In [ ]:
import pandas as pd
import finterstellar as fs

In [ ]:
path = './data/'
cd = 'S&P 500'

In [ ]:
file_name = path + cd + ' Historical Data.csv'
df = pd.read_csv(file_name, index_col='Date')
df.head()

In [ ]:
# 날짜 처리
ld = fs.LoadData()
df = ld.date_formatting(df)    
# date_formatting(데이터프레임) - 날짜 처리
df.head()

In [ ]:
df = ld.price_df_trimming(df, cd)
# price_df_trimming(데이터프레임) - 시세 처리
df.head()

볼린저 밴드 계산

In [ ]:
n = 20     # 평균주가계산 기준일수
sigma = 2     # 편차구간 지정 (시그마의 배수로)

In [ ]:
df['center'] = df[cd].rolling(n).mean()
# 데이터프레임[데이터].rolling(기준일자).mean() - 이동평균 계산
df['ub'] = df['center'] + sigma * df[cd].rolling(n).std()    
# 데이터프레임[데이터].rolling(기준일자).std() - 이동표준편차 계산
df['lb'] = df['center'] - sigma * df[cd].rolling(n).std()
df.tail()

In [ ]:
base_date = '2018-01-01'    # 기준일자 설정

In [ ]:
sample = df[base_date:].copy()    # 기준일 이후 데이터만 가져와 sample에 저장 (샘플링)
sample.head()

트레이드 북 생성

In [ ]:
book = pd.DataFrame()    # 빈 트레이딩북(데이터프레임) 생성
book[cd] = sample[cd]    # sample에서 시세 복사
book['t '+cd] = ''     # 매매전략 입력을 위한 trade 컬럼 생성
book['p '+cd] = ''     # 포지션 입력을 위한 p 종목코드 컬럼 생성
book.head()

트레이딩 전략 수립

In [ ]:
thd = 'ub'   # 종목 매도 기준 (편차구간상단:ub, 편차구간중심:center)

In [ ]:
sample.tail()

In [ ]:
# 백테스팅 매매전략 구현
'''
    sample.loc[i, cd] : i일의 가격
    sample.loc[i, center] : i일의 평균가격 (center)
    sample.loc[i, ub] : i일의 상단가격 (upper bound)
    sample.loc[i, lb] : i일의 하단가격 (lower bound)
'''
for i in sample.index:    # sample에서 날짜를 하나씩 순환하며
    price = sample.loc[i, cd]
    if price > sample.loc[i, thd]:    # 가격 > thd :
        book.loc[i, 't '+cd] = ''    # trade : do nothing
    elif sample.loc[i, thd] >= price and price >= sample.loc[i, 'lb']:    
    # thd >= 가격 & 가격 >= lb 사이에서는:
        if book.shift(1).loc[i, 't '+cd] == 'buy' or book.shift(1).loc[i, 't '+cd] == 'ready':    
        # 이미 매수상태 또는 Ready에서 넘어온 상태
            book.loc[i, 't '+cd] = 'buy'     # trade : buy (매수상태 유지)
        else:
            book.loc[i, 't '+cd] = ''     # trade : clear (zero상태 유지)
    elif sample.loc[i, 'lb'] > price:    # lb > 가격 :if book.shift(1).loc[i, 't '+cd] == 'buy'
        if book.shift(1).loc[i, 't '+cd] == 'buy':
            book.loc[i, 't '+cd] = 'buy'    # 이미 buy
        else:
            book.loc[i, 't '+cd] = 'ready'

트레이딩 전략에 따른 포지션

In [ ]:
status = ''
for i in book.index:
    if book.loc[i, 't '+cd] == 'buy':    # 매수 상태에서는
        if book.shift(1).loc[i, 't '+cd] == 'buy':    # 이미 매수 상태였다면
            status = 'll'     # long -> long 유지
        elif book.shift(1).loc[i, 't '+cd] == '':     # zero 상태였다면
            status = 'zl'     # zero -> long
        else:
            status = 'zl'     # zero -> long
    elif book.loc[i, 't '+cd] == '':
        if book.shift(1).loc[i, 't '+cd] == 'buy':
            status = 'lz'     # long -> zero
        elif book.shift(1).loc[i, 't '+cd] == '':
            status = 'zz'     # zero -> zero
        else:
            status = 'zz'     # zero -> zero
    else:
        status = 'zz'     # zero -> zero
    book.loc[i, 'p '+cd] = status


In [ ]:
book['2018-02-01':'2018-02-20']

In [ ]:
rtn = 1.0
book['return'] = 1
buy = 0.0
sell = 0.0
for i in book.index:

    if book.loc[i, 'p '+cd] == 'zl' or book.loc[i, 'p '+cd] == 'sl' :     # long 진입
        buy = book.loc[i, cd]    # 매수 가격 확정
        print(i.date(), 'long '+cd, buy)
    elif book.loc[i, 'p '+cd] == 'lz' or book.loc[i, 'p '+cd] == 'ls' :     # long 청산
        sell = book.loc[i, cd]    # 매도 가격 확정
        # 손익 계산
        rtn = (sell - buy) / buy + 1    
        # 손익 = (매도가-매수가)/매수가 + 1 , 100원 투자해서 10원 벌면 손익은 1.10
        book.loc[i, 'return'] = rtn    # 트레이딩북에 손익 기록
        print(i.date(), 'long '+cd, buy, ' | unwind long '+cd, sell, ' | return:', round(rtn, 4))

    # 공매도가 허용되는 경우라면
    elif book.loc[i, 'p '+cd] == 'zs' or book.loc[i, 'p '+cd] == 'ls' :     # short 진입
        sell = book.loc[i, cd]    # 공매도 가격 확정
        print(i.date(), 'short '+cd, sell)
    elif book.loc[i, 'p '+cd] == 'sz' or book.loc[i, 'p '+cd] == 'sl' :     # short 청산
        buy = book.loc[i, cd]    # 숏커버 가격 확정
        # 손익 계산
        rtn = (sell - buy) / sell + 1
        book.loc[i, 'return'] = rtn
        print(i.date(), 'short '+cd, sell, ' | unwind short '+cd, buy, ' | return:', round(rtn, 4))

if book.loc[i, 't '+cd] == '' and book.loc[i, 'p '+cd] == '':     # zero position
    buy = 0.0
    sell = 0.0

acc_rtn = 1.0
for i in book.index:
    rtn = book.loc[i, 'return']
    acc_rtn = acc_rtn * rtn
    book.loc[i, 'acc return'] = acc_rtn

print ('Accunulated return :', round(acc_rtn, 4))

In [ ]:
# 벤치마크 수익률
bm_rtn = round (( book[cd].iloc[-1] - book[cd].iloc[0] ) / book[cd].iloc[0] + 1, 4)
print('BM return:', round(bm_rtn, 4) )

In [ ]:
exs_rtn = ( round(acc_rtn/bm_rtn, 4) - 1 ) * 100
print('Excess return:', round(exs_rtn, 4) )

그래프를 그려봅시다. 그래프는 fs를 이용해서...

In [ ]:
v = fs.Visualize()   # fs 라이브러리의 Visualize() 모듈을 불러와 v로 지정

In [ ]:
v.BB_trend_view(sample, cd)
# v 모듈의 bb_trend_view(데이터프레임, 편차구간, 종목코드, (사이즈)) 함수를 이용해 볼린저밴드 그래프 그리기

In [ ]:
v.position_view(book, [cd])
# v 모듈의 position_view(트레이딩북, 종목코드, (사이즈)) 함수를 이용해 포지션 보유내역 그래프 그리기